In [1]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
# 数值特征
import pandas as pd
import numpy as np
import json
from tqdm import tqdm as tqdm
from sklearn.model_selection import KFold
from gensim.models import word2vec
from gensim.models import Word2Vec
from sklearn.metrics import mean_squared_error as mse

In [2]:
data = pd.read_pickle('data/step1.pkl')
test_data = pd.read_pickle('data/step1_test.pkl')

In [8]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item

name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = sorted(list(set(name_id[id])))

In [9]:
mention_vector = np.load('model_type/type_vector.npy')

In [27]:
type_vector = {}
c = 0
for s in train_data:
    mention_ner = s['mention_data']
    for m in mention_ner:
        if m['kb_id'] == 'nil':
            continue
        name_list = [kb[m['kb_id']]['subject']] + kb[m['kb_id']]['alias']
        if m['mention'] not in name_list:

            # kb[m['kb_id']]['alias'] += [m['mention']]
            keep = False
            for name_c in ''.join(name_list):
                if name_c in m['mention']:
                    # print(name_c,m['mention'])
                    keep = True
                    break

            if not keep:
                continue
        
        
        type_vector[s['text_id']+m['mention']] = mention_vector[c]
        c += 1


In [30]:
from data_prepare import data_manager,read_kb

data_manager.read_basic_info()
type_list = data_manager.type_list

In [31]:
np.argmax([1,2,3])

2

In [33]:
def cal_type_vector(kbid,textid,mention):
    spe_type = type_list.index(kb[kbid]['type'][0])
    if textid+mention in type_vector:
        loss = 1-type_vector[textid+mention][spe_type]
        max_equal = np.argmax(type_vector[textid+mention])==spe_type
        return loss,max_equal
    else:
        return None,None

In [34]:
type_vector_arr = data.apply(lambda x:cal_type_vector(x.kb_id,x.text_id,x.train_mention),axis=1)

In [37]:
data['type_loss'] = [x[0] for x in type_vector_arr]
data['type_equal'] = [x[1] for x in type_vector_arr]

In [38]:
data.to_pickle('data/step6.pkl')

In [3]:

# 引入embedding特征


In [4]:
mention_vector.shape

(256200, 49)

In [5]:
data.shape

(1566680, 38)